## 1. Hyperparametersuche

Fast alle Machine Learning Modelle verfügen über sogenannte Hyperparamter. Dies sind veränderliche Bestandteile des Modells bzw. des Trainings im weiteren Sinne (also auch Vorverarbeitungen der Daten), die nicht durch die Daten bestimmt werden können. Dies ist im Kontrast zu Parametern wie zum Beispiel den Gewichten in einem Neuronenmodell, die eben genau durch die Daten bestimmt werden.

Hyperparameter müssen abgestimmt werden ohne dabei auf die Testdaten zurückzugreifen. Werden Testdaten zur Abstimmung verwendet, nennt man diesen methodischen Fehler *data leakage* - also das Einsickern der Testdaten in den Lernprozess. Damit einher geht eine möglicherweise falsch geschätzte Generalisierungsfähigkeit des Modells - die Testdaten sind keine Testdaten mehr.

Der korrekte Ansatz zur Abstimmung der Hyperparameter ist die Benutzung eines Validierungsdatensatzes. Ein Teil der Trainingsdaten wird zur Validierung von Modellen mit verschiedenen Hyperparametern genutzt - die Testdaten bleiben unangetastet.

Folgende Hyperparameter haben wir kennengelernt:

- Lineare Regression
    - `Ridge` versus `Lasso` versus `ElasticNet` - Art der Regularisierung
    - `alpha` - Stärke der Regularisierung
    - `degree` - Grad einer möglichen polynomischen Expansion
- Logistische Regression
    - `C` - inverse Stärke der Regularisierung
    - `penalty= "l2" / "l1" / "elasticnet"`- Art der Regularisierung
- Decision Trees
    - `max_depth`
- Random Forest
    - `max_depth`
    - `n_estimators`
- Gradient Boosting
    - `max_depth`
    - `n_estimators`
    - `learning_rate`
- Support Vector Machines
    - `C` - inverse Stärke der Regularisierung
    - `kernel = "rbf" / "linear"`
    - `gamma`
- K-Nearest Neighbors
    - `n_neighbors`


### 1.1 Daten und Vorbereitung

Wir untersuchen ein künstlich erstelltes Regressionsproblem.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# TODO: Scikit-Learn Importe

In [2]:
# TODO: Hilfsfunktion zum Erstellen einer polynomischen Regression
def get_regression(alpha=1.0, degree=2):
    model = None
    return model

In [3]:
# Hilfsfunktion zu Generierung eines Toy-Datensatzes
# für ein Regressionsproblem
def make_data(N=30, err=0.8, rseed=1):
    # randomly sample the data
    rng = np.random.RandomState(rseed)
    X = rng.rand(N, 1) ** 2
    y = 10 - 1. / (X.ravel() + 0.1)
    if err > 0:
        y += err * rng.randn(N)
    return X, y

In [4]:
# TODO: Visualisierung der Daten
# shape [N, 1] für X und [N, ] für y
X, y = make_data(N=100)

In [10]:
# TODO: Trainings- und Testdaten aufspalten

In [5]:
# TODO: Fitten des Modells ohne Hyperparametersuche
# TODO: Evaluieren

### 1.2. Hyperparametersuche per Scikit-Learn API

Scikit-Learn stellt zur Abstimmung der Hyperparameter verschiedene Klassen zur Verfügung. Diese Klassen führen die nötigen for-Schleifen intern aus. 

![gridsearch_cv](https://raw.githubusercontent.com/layerwise/training/main/assets/gridsearch_cv.png)

Die obenstehende Graphik ist durch `GridSearchCV` von Scikit-Learn umgesetzt.

```python
>>> from sklearn.model_selection import GridSearchCV
>>> GridSearchCV?
```

Wir müssen von außen nur folgendes festlegen: 

- `param_grid` - den Suchraum der Hyperparameter
- `model` - das zu Grunde liegende Modell
- `cv` - die Anzahl der Kreuzvalidierungs-Faltungen, typischerweise 3, 5 oder 10
- `scoring` - die zu optimierende Metrik

Um alle möglichen Evaluationsmetriken anzuzeigen ist folgender Code hilfreich

```python
>>> from sklearn.metrics import SCORERS
>>> sorted(SCORERS.keys())
```

Die Klasse `GridSearchCV` verhält sich ganz im Sinne von Scikit-Learn ganz genau so wie jedes andere Basis-Modell auch. Das heißt es gibt dort eine `fit` und `predict` Funktion, in denen alle Arbeitsschritte schon integriert sind.

Nach einem erfolgreichen Fit, das heißt einer Abstimmung der Hyperparameter durch `GridSearchCV` sind unter anderem folgende Attribute relevant:

- `best_estimator_`
- `best_params_`
- `best_score_`
- `cv_results_`

In [6]:
# TODO: Import GridSearchCV
# TODO: Fitten des GridSearchCV Modells
# TODO: Evaluieren

### 1.3 Hyperparametersuche durch vorgefertigte Modelle

Die Klasse `GridSearchCV` ist für beliebige Modelle (und besonders für Pipelines) nutzbar. Für viele Modelle allerdings ist die Abstimmung von Hyperparametern so alltäglich, dass es spezialisierte Klassen gibt.
Diese sind:
- `RidgeCV`
- `LassoCV`
- `ElasticNetCV`
- `LogisticRegressionCV`

Diese setzen exakt das um, was `GridSearchCV` auch könnte, also sind folgende Codes identisch

```python
>>> model = GridSearchCV(
...     Ridge(),
...     param_grid={"alpha": [0.01, 0.1, 1.0]})
>>> model.fit(X, y)
```

und

```python
>>> model = RidgeCV(alphas=[0.01, 0.1, 1.0])
>>> model.fit(X, y)
```

![gridsearch_cv](https://raw.githubusercontent.com/layerwise/training/main/assets/gridsearch_cv.png)

In [ ]:
# TODO: Import RidgeCV
# TODO: Fit
# TODO: Evaluieren

### 1.4. Hyperparametersuche durch Zufallssuche

Im Allgemeinen ist die sogenannte erschöpfende Gittersuche nicht nötig und auch nicht empfehlenswert, wie anhand folgender Graphik erkennbar. Dann kann statt `GridSearchCV` besser `RandomizedSearchCV` verwendet werden.

![gridsearch_vs_randomsearch](https://raw.githubusercontent.com/layerwise/training/main/assets/gridsearch_vs_randomsearch.png)


In [7]:
# TODO: Import RandomizedSearchCV
# TODO: Fit
# TODO: Evaluieren

## 2. Kreuzvalidierung

Die Kreuzvalidierung kann nicht nur zur Abstimmung von Hyperparametern verwendet werden. Tatsächlich ist ihre vornehmliche Funktion, die Generalisierungsfähigkeit des Modells besser zu schätzen. Statt eines einzigen Testdatensatzes, der immer zufälligen Schwankungen ausgesetzt ist und damit zu einer zufälligen Schätzung der Generalisierungsfähigkeit führt, benutzt die Kreuzvalidierung 3, 5 oder 10 solche Testdatensätze. Am Ende wird aus den einzelnen Schätzungen ein Mittelwert gebildet, von dem man zeigen kann, dass er die tatsächliche Generalisierung genauer schätzt. Wir sind also vor über-optimistischen und unter-pessimistischen Schätzungen gefeit. Wann immer genug Rechenleistung vorhanden ist, ist die Kreuzvalidierung der Testdaten-Methode vorzuziehen.

In Scikit-Learn lässt sich die Kreuzvalidierung mit `cross_val_score` umsetzen. Wichtig zu beachten ist, dass ein dedizierter Trainings-Test-Split hier **nicht** erforderlich ist. Der Funktion `cross_val_score` wird der Gesamtdatensatz übergeben.

![crossvalidation](https://raw.githubusercontent.com/layerwise/training/main/assets/crossvalidation.png)

In [8]:
# TODO: Import cross_val_score
# TODO: Evaluation mit Kreuzvalidierung

## 3. Verschachtelte Kreuzvalidierung

Die verschachtelte Kreuzvalidierung nutzt die Methode der Kreuzvalidierung gleichzeitg sowohl für die Abstimmung von Hyperparametern und zur besseren Schätzung der Generalisierungsfähigkeit. In Scikit-Learn ist die Umsetzung denkbar einfach: wir verbinden einfach `GridSearchCV` mit `cross_val_score`.

![nested_crossvalidation](https://raw.githubusercontent.com/layerwise/training/main/assets/nested_crossvalidation.png)

In [9]:
# TODO